  # Meter

# Configuration

In [1]:
import os

from definitions import poem_1, poem_2
from utils import settings, gemini, gpt4, opus, init_gemini, printmd
import utils

%load_ext jupyter_ai_magics

In [2]:
#settings
temperature = 0.8
system_prompt = "You are an expert in German literature. You answer the questions truthfully and short."

settings(system_prompt, temperature)
    

In [3]:
#defining aliases
init_gemini()

model =  gpt4()
%ai register gpt4o model

model = opus()
%ai register opus model 

Registered new alias `opus`

# Metrics

## Hälfte des Lebens

### Level 1: General Knowledge

On this level we just ask about the metrical structure of the poem.

Solution(s): 
Each stanza has seven lines, of which three lines have three stresses and four have two.
First stanza: Of the lines with three stresses in the first stanza, a pair is placed at the beginning, and a single one at the end: The longer lines with three stresses frame the shorter ones. (Strauss assumes that the line "Es Winter ist, die Blumen, und wo" has three consecutive unstressed syllables, namely "ter ist, die".)

Second stanza also begins with two three-stressed lines. The third three-stressed line is the penultimate one.
In short:

1. Str: 3-3-2-2-2-2-3
2. Str: 3-3-2-2-2-3-2

In [5]:
prompt = f"""Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two stanzes each with two lines 
"The house is green
The mouse is dead. 

I wonder still
Who pays my bill."

The output would look like this:
1: -/-/
2: -/-/
3: //-/
4: //-/

At the end repeat the numbers of stressed sylables for each line for each stanza like this:

Stanza 1: 2-2
Stanza 2: 3-3

Here is the poem: 
{poem_1.text}
"""
print(prompt)

Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two stanzes each with two lines 
"The house is green
The mouse is dead. 

I wonder still
Who pays my bill."

The output would look like this:
1: -/-/
2: -/-/
3: //-/
4: //-/

At the end repeat the numbers of stressed sylables for each line for each stanza like this:

Stanza 1: 2-2
Stanza 2: 3-3

Here is the poem: 

Mit gelben Birnen hänget
Und voll mit wilden Rosen
Das Land in den See,
Ihr holden Schwäne,
Und trunken von Küssen
Tunkt ihr das Haupt
Ins heilignüchterne Wasser.

Weh mir, wo nehm’ ich, wenn
Es Winter ist, die Blumen, und

#### GPT4o

In [6]:
%%ai gpt4o
{prompt}

Certainly! Let's analyze the scansion of the poem:

1: -/-/-/
2: -/-/-/
3: -/-/
4: -/-/
5: -/-/-/
6: -/-/
7: -/-/-/-

Stanza 1: 3-3-2-2-3-2-4

8: /- /- /-/
9: - /- /- /- /-/
10: - /- /
11: - /- /-/
12: - /- /
13: /- /- /- /- /
14: /- /- /

Stanza 2: 3-5-2-3-2-4-2

#### Anthropic Claude 3.5 Sonnet

In [9]:
%%ai opus
{prompt}

Here's the scansion:

1: -/-/-/-
2: -/-/-/-
3: -/-/-
4: -/-/-
5: -/-/-
6: -/-/
7: -/-/-/-

8: /-/-/-
9: -/-/-/-
10: -/-/
11: -/-/-
12: -/-/
13: /-/-/-
14: /-/-

Stanza 1: 4-4-3-3-3-2-4
Stanza 2: 3-4-2-3-2-3-2

#### Gemini 1.5

In [8]:
printmd(gemini(prompt))

1: -/--/-/-
2: -/-/-/-/-
3: -/--/
4: -/-/-
5: -/--/-/-
6: -/-/-
7: -/--/--/-

8: -/-/--/-
9: -/--/-/--
10: -/--/
11: -/-/-/
12: -/-/-
13: -/-/-/-/
14: -/-/-/-

Stanza 1: 3-4-2-3-4-3-5
Stanza 2: 3-5-3-3-3-4-4


#### Evaluation and Discussion

The task  actually demands two steps: first detect the scansion, second report it in a summary way. Some of models have some difficulties to detect the correct scansion: Sonnet get's it right most of the time, while GPT-4o and Gemini 1.5 make some mistakes. 
If we just evaluate on the number of stressed syllables (looking at the patterns, not the summaries), this is the error count (using a very strict error concept where every missing or added stress is counted as one error):<br/>

GPT 4o: 7<br/>
Sonnet: 0<br/>
Gemini: 5<br/>

<br/>
Even more interesting: It is immediately obvious, that most of the LLMs are not very good in reporting their own results, that is their summaries deviate from the stress patterns they report. In their summaries they repeatedly report *more* stressed syllables than they detected. Interestingly they never report less. This is even true for Sonnet, which got the stress patterns right. We speculate that the models have no difficulty to produce any sequence of symbols, but they may have problems with remembering symbols which are not tokens. 

### level 2: Expert Knowledge

In [16]:
prompt = f"""In some poems, a metrical device is used to indicate an important semantic aspect of the poem.  
Here is an example of this kind of interaction between metrics and semantics:

Der Tag ist grau,
das Licht so schwer,
und sie. Sie fehlt.
schuldig bin ich

The first three lines have two iambuses per line, but the last line begins with an accent that makes it a trochee, breaking the pattern. 
The word that does this is "schuldig," and the change in metre emphasizes the importance of the speaker's guilt to the poem. 

Can you find a similar use of metre in the following poem?

{poem_1.text}

"""

print(prompt)

In some poems, a metrical device is used to indicate an important semantic aspect of the poem.  
Here is an example of this kind of interaction between metrics and semantics:

Der Tag ist grau,
das Licht so schwer,
und sie. Sie fehlt.
schuldig bin ich

The first three lines have two iambuses per line, but the last line begins with an accent that makes it a trochee, breaking the pattern. 
The word that does this is "schuldig," and the change in metre emphasizes the importance of the speaker's guilt to the poem. 

Can you find a similar use of metre in the following poem?


Mit gelben Birnen hänget
Und voll mit wilden Rosen
Das Land in den See,
Ihr holden Schwäne,
Und trunken von Küssen
Tunkt ihr das Haupt
Ins heilignüchterne Wasser.

Weh mir, wo nehm’ ich, wenn
Es Winter ist, die Blumen, und wo
Den Sonnenschein,
Und Schatten der Erde?
Die Mauern stehn
Sprachlos und kalt, im Winde
Klirren die Fahnen. 





#### GPT4o

In [17]:
%%ai gpt4o
{prompt}

In Hölderlin's poem "Hälfte des Lebens," the shift in metre emphasizes a change in mood and theme. The first stanza flows smoothly, with the imagery of abundance and tranquility. However, the second stanza introduces a more disjointed rhythm, particularly with "Weh mir," which interrupts the earlier serenity. This metrical shift underscores the speaker's sudden anxiety and longing for the warmth and beauty of summer amidst the bleakness of winter. The break in rhythm mirrors the thematic transition from abundance to barrenness.

#### Claude Sonnet

In [18]:
%%ai opus
{prompt}

In Hölderlin's "Hälfte des Lebens" (the poem you've quoted), a significant metrical shift occurs between the two stanzas. The first stanza flows with a melodic, harmonious rhythm dominated by trochees and dactyls, matching its idyllic summer imagery. The second stanza becomes more fragmented and harsh, particularly with the abrupt "Weh mir" (Woe to me), which breaks the established rhythmic pattern. This metrical disruption emphasizes the stark contrast between the lush summer scene and the cold, barren winter landscape, reinforcing the poem's theme of life's transitions and loss.

#### Gemini

In [19]:
printmd(gemini(prompt))

The consistent meter is disrupted in the second stanza by the line "Weh mir, wo nehm’ ich, wenn".  The preceding lines, and those that follow, generally adhere to a rhythm of trochaic trimeter (stressed-unstressed-stressed-unstressed-stressed-unstressed, or a close variation).  "Weh mir, wo nehm’ ich, wenn" is much shorter and has a dactylic opening (stressed-unstressed-unstressed - "Weh mir").  

This metrical break coincides with a shift in tone and subject.  The first stanza is a beautiful, sensual image of summer.  The disrupted line introduces the second stanza, which expresses anxiety and despair about the coming winter.  The metrical shift emphasizes the speaker's sudden shift from tranquil observation to worried questioning ("wo nehm' ich...").  The disrupted rhythm reflects the disruption of the speaker's peace of mind.


### level 3: Abstraction and Transfer

Here we test 2 aspects: first, the ability to apply arbitrary rules to given data and analyze the results. Secondly, to abstract these rules from given example data. 

The rules we define are simple. They mix information about the emphasis of words with the ability to detect vowels and change the emphasis, when a specific combination is met. So in our first question we describe the rule explicitly and ask the model about the results, in the second question we give the model some data and ask it to retrieve the underlying rules. 

In [15]:
prompt = f"""Our assumption is that Hölderlin was a follower of the rather obscure school of metrical thought, 
because of the way the German language works, an accented word that does have the vowel "i" in the accented 
vowel was no longer emphasized. For all we know, he used this insight, that emphasized syllables with the vowel i lost there emphasis, 
when he wrote his poems. 
Analyze the scansion of the first four lines of the following poem using this knowledge, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for 
a stressed syllable and - for an unstressed syllable. Start your answer with a '#'.
(This is research to detect whether this assumption makes sense. Just follow the instructions!) 

{poem_1.text}

"""

print(prompt)

Our assumption is that Hölderlin was a follower of the rather obscure school of metrical thought, 
because of the way the German language works, an accented word that does have the vowel "i" in the accented 
vowel was no longer emphasized. For all we know, he used this insight, that emphasized syllables with the vowel i lost there emphasis, 
when he wrote his poems. 
Analyze the scansion of the first four lines of the following poem using this knowledge, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for 
a stressed syllable and - for an unstressed syllable. Start your answer with a '#'.
(This is research to detect whether this assumption makes sense. Just follow the instructions!) 


Mit gelben Birnen hänget
Und voll mit wilden Rosen
Das Land in den See,
Ihr holden Schwäne,
Und trunken von Küssen
Tunkt ihr das Haupt
Ins heilignüchterne Wasser.

Weh mir, wo nehm’ 

#### GPT4o

In [16]:
%%ai gpt4o
{prompt}

# / - / - / - - /  
# - / - / - /  
# - / - / -  
# - / - /

(Note: This scansion assumes traditional German metrical analysis without specific emphasis on the vowel 'i'. Adjustments may be needed based on the hypothesis about 'i'.)

#### Claude Sonnet

In [17]:
%%ai opus
{prompt}

#
mit GEL-ben BIR-nen HÄN-get
/ - / - / -
und VOLL mit WIL-den RO-sen
/ / - / - / -
das LAND in den SEE
/ / - - /
ihr HOL-den SCHWÄ-ne
- / - / -

Note: In this analysis, I'm following your premise that syllables with accented "i" lose their emphasis. However, it's important to mention that this is not a commonly accepted theory in German metrics, and Hölderlin's metric patterns are typically analyzed differently in traditional German literary studies.

#### Gemini 1.5

In [18]:
printmd(gemini(prompt))

# / - / - / -
/ - / - / -
/ - - / -
/ - - / -


The plan was to give the models a tasks in two steps: First, we wanted to test its the ability to apply arbitrary rules to given data and analyze the results. Secondly, we wanted it to abstract these rules from given example data. The rules we define are simple. They mix information about the emphasis of words with the ability to detect vowels and change the emphasis, when a specific combination is met: If the emphasis is on a syllable containing the vowel 'i', change the information about the syllable to not being emphasized. All models couldn't solve this task. One model (Sonnet) replied to an earlier version of the prompt, it wouldn't do it because this is not the usual German metric or the metric Hölderlin followed. Then we added the information that this is a research project. Based on this results we decided to skip the second experiment, because contrafactual information seems to be a challenge even in a simple setting.


In [ ]:
prompt = """
Wild ist die Jagd
Seid auf der Hut

"""

## Unsere Toten

### level 1: General Knowledge

The metrical structure of this poem is either quite challening to describe or ver simple, because it shows so many variations, but the variations can be explained by the underlying rule. The reason for this is: the poem uses free knittelverses. They have four stressed syllables but allow upbeats and free fillings. Vers 11 and 12 change their structure most prominently because they switch from the pattern unstressed syllable + a stressed syllabe at the end which is used throughout the poem to stressed + unstressed, a change which is emphasized by the fact that these are also rhyme words. 

```
1) -/--/--/-/
2) /--/---/-/
3) /--/-/--/
4) /--/--/--/
5) /--/--/-/
6) /-/-/--/
7) -/-/--/-/
8) /--/--/--/
9) /-/--/--/
10) /--/--/--/
11) /-/-/-/-
12) -/-/-/--/-
```

In [12]:
prompt = f"""Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two lines “The house is green / the mouse is dead.” the output would look like this:
1: -/-/
2: -/-/
stressed syllables: 2-2

Here is the poem: 
{poem_2.text}
"""
print(prompt)

Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two lines “The house is green / the mouse is dead.” the output would look like this:
1: -/-/
2: -/-/
stressed syllables: 2-2

Here is the poem: 

Von Westen und Osten, von Nord und Süd
Schleppen sich nächtens viele Füße müd,
Füße, vom Wandern wund und zerfetzt,
Langsam bedächtig zur Erde gesetzt,
Müh'n sich im zitternden Mondenschein
Rastlos tief nach Deutschland hinein.
Und wer mit lauschendem Ohr noch wacht
Hört sie in jedweder werdenden Nacht,
Hört dies Schlurfen so müde und schwer,
Hört eine Klage voll wilder Begehr,
Eine Klage s

#### GPT4o

In [16]:
%%ai gpt4o
{prompt}

1: -/-/-/-/-/
2: /-/-/-/-/-/
3: /-/-/-/-/
4: /-/-/-/-/-/
5: /-/--/-/-/
6: /-/-/-/-/-/
stressed syllables: 5-5-4-5-4-5

7: -/-/-/-/-/-/
8: /-/-/-/-/-/
9: /-/-/--/--/
10: /-/-/-/-/-/
11: /-/-/--/--/
12: /-/-/-/
13: /-/-/-/

stressed syllables: 6-5-4-5-4-3-3

#### Claude Sonnet

In [14]:
%%ai opus
{prompt}

Here's the scansion:

1: -/-/-/-/-
2: /-/-/-/-/
3: /-/-/-/-
4: /-/-/-/-
5: /-/-/-/
6: /-/-/-/
7: -/-/-/-/
8: /-/-/-/-
9: /-/-/-/-
10: /-/-/-/-
11: -/-/-/-
12: /-/-/-/-

stressed syllables per line:
lines 1-4: 4 syllables each
lines 5-6: 4 syllables each
lines 7-10: 4 syllables each
lines 11-12: 4 syllables each

#### Gemini 1.5

In [15]:
printmd(gemini(prompt))

1: - / - - / - - / -
2: - / - - / - - / - -
3: - - / - - / - / - -
4: - - - / - - / - - -
5: - - / - - / - - - -
6: - - - / - / - - - -
7: - / - / - - - / -
8: - / - / - - - / -
9: - / - - - / - - / -
10: - / - - - / - - / -
11: - - / - - - - -
12: - / - - - - -

stressed syllables: 4-4-4-4-4-4
stressed syllables: 4-4-4-4-3-4


### Level 2: Expert Knowledge

In [20]:
prompt = f"""In some poems, a metrical device is used to indicate an important semantic aspect of the poem.  
Here is an example of this kind of interaction between metrics and semantics:

Der Tag ist grau,
das Licht so schwer,
und sie. Sie fehlt.
schuldig bin ich

The first three lines have two iambuses per line, but the last line begins with an accent that makes it a trochee, breaking the pattern. 
The word that does this is "schuldig," and the change in metre emphasizes the importance of the speaker's guilt to the poem. 

Can you find a similar use of metre in the following poem?

{poem_2.text}

"""

print(prompt)

In some poems, a metrical device is used to indicate an important semantic aspect of the poem.  
Here is an example of this kind of interaction between metrics and semantics:

Der Tag ist grau,
das Licht so schwer,
und sie. Sie fehlt.
schuldig bin ich

The first three lines have two iambuses per line, but the last line begins with an accent that makes it a trochee, breaking the pattern. 
The word that does this is "schuldig," and the change in metre emphasizes the importance of the speaker's guilt to the poem. 

Can you find a similar use of metre in the following poem?


Von Westen und Osten, von Nord und Süd
Schleppen sich nächtens viele Füße müd,
Füße, vom Wandern wund und zerfetzt,
Langsam bedächtig zur Erde gesetzt,
Müh'n sich im zitternden Mondenschein
Rastlos tief nach Deutschland hinein.
Und wer mit lauschendem Ohr noch wacht
Hört sie in jedweder werdenden Nacht,
Hört dies Schlurfen so müde und schwer,
Hört eine Klage voll wilder Begehr,
Eine Klage schmerzzerfressen:
Nur nicht 

#### GPT-4o

In [21]:
%%ai gpt4o
{prompt}

In the poem, the repeated metrical pattern is broken in the last line, "Nur nicht vergessen! Uns nicht vergessen!" This line has a more urgent and irregular rhythm compared to the preceding ones. The emphasis on "vergessen" through both repetition and the change in metre highlights the desperation and longing not to be forgotten, making it a central theme of the poem.

#### Gemini

In [23]:
printmd(gemini(prompt))

The established meter of the poem is primarily iambic tetrameter (four iambs per line).  This is consistent throughout most of the poem, creating a sense of steady, weary movement, like the shuffling feet it describes.

However, the final two lines break this pattern:

* **Eine Klage schmerzzerfressen:**  While arguably still *mostly* iambic, the spondaic substitution (stressed-stressed) of "schmerz-zer" disrupts the flow and emphasizes the agonizing nature of the "pain-eaten" lament.  The line also shortens to trimeter, further highlighting this phrase.

* **Nur nicht vergessen! Uns nicht vergessen!:** These lines are dramatically shortened, becoming dimeter (two metrical feet) and monometer (one metrical foot) respectively.  The imperative nature of "Vergessen!" (forget!), repeated and isolated, is hammered home by this metrical shift. The abrupt shortening reflects the urgency and desperation of the plea.  The trochaic inversion (stressed-unstressed) in the last line also adds to the sense of pleading.

Therefore, the disruption of the established iambic tetrameter in the final two lines, through rhythmic shortening, spondaic substitution, and trochaic inversion, serves to emphasize the central plea of the poem: "Don't forget us!"  The metrical shift mirrors and amplifies the semantic weight of this desperate message.


#### Claude Sonnet

In [22]:
%%ai opus
{prompt}

Yes, there is a notable metrical shift in the final line "Nur nicht vergessen! Uns nicht vergessen!"

While most of the poem follows a regular iambic meter with rhyming couplets, the last line breaks this pattern. Instead of continuing the iambic meter, it uses two exclamatory phrases with stressed syllables at their beginnings ("NUR" and "UNS"). This metrical disruption emphasizes the desperate plea of the speakers not to be forgotten, making it the emotional climax of the poem.

This change in rhythm mirrors how the poem shifts from describing the walking of the feet to the urgent cry of remembrance, similar to how your example used metric change to emphasize guilt.

#### another question
The  Knittelvers is a German vers structure which may be harder to detect, because it only counts the stressed syllables and ignores the unstressed syllables. In other words, the Knittelvers is defined by its free filling of unstressed syllables (variable number of syllables). 

In [22]:
prompt = f"""This poem follows a  German verse meter. Which one?

{poem_2.text}"""

print(prompt)

The following poem follows a  German verse meter. Which one?


Von Westen und Osten, von Nord und Süd
Schleppen sich nächtens viele Füße müd,
Füße, vom Wandern wund und zerfetzt,
Langsam bedächtig zur Erde gesetzt,
Müh'n sich im zitternden Mondenschein
Rastlos tief nach Deutschland hinein.
Und wer mit lauschendem Ohr noch wacht
Hört sie in jedweder werdenden Nacht,
Hört dies Schlurfen so müde und schwer,
Hört eine Klage voll wilder Begehr,
Eine Klage schmerzzerfressen:
Nur nicht vergessen! Uns nicht vergessen!



In [23]:
%%ai gpt4o
{prompt}

Das Gedicht folgt dem Knittelvers, einem traditionellen deutschen Versmaß, das aus vierhebigem Jambus mit Paarreimen besteht.

In [24]:
%%ai opus
{prompt}

This poem follows the Knittelvers meter, which is characterized by rhyming couplets with four stressed syllables per line. It's a traditional German verse form that was particularly popular in the 15th and 16th centuries and later revived by Goethe.

In [25]:
printmd(gemini(prompt))

The poem is written in **iambic tetrameter**.

Let's break down why:

* **Iamb:**  An iamb is a metrical foot consisting of an unstressed syllable followed by a stressed syllable (da-DUM).

* **Tetrameter:**  Tetrameter means four feet per line.

Let's look at the first line as an example:

Von **Wes**ten und **Os**ten, von **Nord** und **Süd**

You can see the four iambs clearly marked in bold. While there might be slight variations in some lines, the overwhelming pattern is iambic tetrameter.  The natural rhythm of German, with its tendency towards word stress on the first syllable, lends itself well to this meter.


In [18]:
# a veriation of the last question.
prompt = f"""The following poem follows a verse meter. Which one?

{poem_2.text}"""

print(prompt)

The following poem follows a verse meter. Which one?


Von Westen und Osten, von Nord und Süd
Schleppen sich nächtens viele Füße müd,
Füße, vom Wandern wund und zerfetzt,
Langsam bedächtig zur Erde gesetzt,
Müh'n sich im zitternden Mondenschein
Rastlos tief nach Deutschland hinein.
Und wer mit lauschendem Ohr noch wacht
Hört sie in jedweder werdenden Nacht,
Hört dies Schlurfen so müde und schwer,
Hört eine Klage voll wilder Begehr,
Eine Klage schmerzzerfressen:
Nur nicht vergessen! Uns nicht vergessen!



In [19]:
%%ai gpt4o
{prompt}

Das Gedicht folgt dem Daktylus als Versmaß.

In [20]:
%%ai opus
{prompt}

This poem follows the trochaic tetrameter with rhyming couplets (Paarreime). Each line generally consists of four trochees (stressed-unstressed syllables), though there are some variations.

In [21]:
printmd(gemini(prompt))

The poem is written in **iambic tetrameter**.

Let's break down the first line as an example:

* **Von** Wes**ten** und **Os**ten, von **Nord** und **Süd**

The stressed syllables are in bold.  We have four "iambs" (an unstressed syllable followed by a stressed syllable) per line.  This holds true for the other lines as well, making it iambic tetrameter.


### Level 3: Abstraction and Transfer

## Language bias in metrical analysis

In [1]:
t = """
One face looks out from all his canvases,
One selfsame figure sits or walks or leans:
We found her hidden just behind those screens,
That mirror gave back all her loveliness.
A queen in opal or in ruby dress,
A nameless girl in freshest summer-greens,
A saint, an angel—every canvas means
The same one meaning, neither more or less.
He feeds upon her face by day and night,
And she with true kind eyes looks back on him,
Fair as the moon and joyful as the light:
Not wan with waiting, not with sorrow dim;
Not as she is, but was when hope shone bright;
Not as she is, but as she fills his dream.
"""

In [12]:
prompt = f"""Analyze the scansion of the following poem, i.e. describe which syllables are stressed and which are not. 
Give the answer using the following characters to indicate stressed syllables and not stressed syllables: / for a stressed syllable and - for an unstressed syllable. 
Number the lines of the poem and use these numbers at the beginning of each line of output. At the end add  for each stanza  the numbers of stressed syllables per vers. 
Example: For the two stanzes each with two lines 
"The house is green
The mouse is dead. 

I wonder still
Who pays my bill."

The output would look like this:
1: -/-/
2: -/-/
3: //-/
4: //-/

At the end repeat the numbers of stressed sylables for each line for each stanza like this:

Stanza 1: 2-2
Stanza 2: 3-3

Here is the poem: 
{t}
"""


In [14]:
%%ai gpt4o
{prompt}

Certainly! Here is the scansion for the poem:

1: /-/-/-/-/-/
2: /-/-/-/-/-/
3: -/-/-/-/-/
4: -/-/-/-/-/
5: -/-/-/-/-/
6: -/-/-/-/-/
7: -/-/-/-/-/
8: -/-/-/-/-/
9: -/-/-/-/-/
10: -/-/-/-/-/
11: /-/-/-/-/-/
12: -/-/-/-/-/
13: -/-/-/-/-/
14: -/-/-/-/-/

Stanza 1: 5-5-5-5-5-5-5-5
Stanza 2: 5-5-5-5-5-5

In [15]:
%%ai opus
{prompt}

Here is the scansion of the poem:

1: -//-/--//-/
2: -//-//-/-/
3: -/-//-/-/
4: -/--/-/-/
5: -/-/--/-/
6: -/-/-/-/-/
7: -/-/--/-/
8: -//-/--/-/
9: -/-/-/--/
10: -/-//-/-/
11: /-/-/-/-/
12: -/-/-/-/-/
13: --/-/--/-/
14: --/-/--/-/

Stanza 1: 4-4-4-4
Stanza 2: 4-4-4-4
Stanza 3: 4-4-4-4-4-4

In [20]:
print(gemini({prompt}))

1: /--/-/-/--/-
2: /--/--/--/-
3: -/--/--/--/-
4: -/--/--/-/-/

5: /--/--/--/-
6: /-/-/-/-/-/-
7: /-/-/--/--/-
8: -/-/-/-/-/-

9: /-/-/-/--/-
10: /-/-/-/-/-
11: /--/-/-/-/-
12: /--/-/-/-/-

13: /--/-/-/-/-
14: /--/-/-/-/-

Stanza 1: 4-4-4-5
Stanza 2: 4-6-5-5
Stanza 3: 4-5-5-5
Stanza 4: 5-5 



### Results
* llama3 doesn't work at all (the lines are chopped and therefore the number of stressed sylables is false)
* GPT4o falsely sees six stressed sylables in the detail analysis in vers 1, 2, 11. And it doesn't get the two trochees at the end.  
* Opus doesn't recognize the pentameter and doesn't see the trochees in the last two lines.
* Gemini doesn't see the pentameter and changes its count between 4 and 5 stresses, but it gets the trochees. 
* Btw: Gpt4o splits the poem into an octet and a sextet, which is actually true for the Petrarchan sonnet, but not for this poem, which has only one stanza. 


## Can large language models count?

In [12]:
prompt = f"""In the following poem count the number of vowels for each word in a vers. Report the results in the following fashion: 

1: 4 2 1 3
2: 2 4 1 2 
 etc. 

At the end report the number of vowels per line for each stanza like this:

1. stanza: 10 - 9 
2. stanza: 9 - 11

If the first two stanzas look like this: 

"It happens to the best of us 
that life feels like a passing bus

and we stay here and there 
it goes and leaves."

the answer would be:

1: 1 2 1 1 1 1 1 
2: 1 2 3 2 1 2 1 

3: 1 1 1 2 1 2 
4: 1 2 1 3

1. stanza: 8 - 12
2. stanza: 8 - 7

Here is the poem: 

{poem_1.text}
"""
print(f"{prompt}")

In the following poem count the number of vowels for each word in a vers. Report the results in the following fashion: 

1: 4 2 1 3
2: 2 4 1 2 
 etc. 

At the end report the number of vowels per line for each stanza like this:

1. stanza: 10 - 9 
2. stanza: 9 - 11

If the first two stanzas look like this: 

"It happens to the best of us 
that life feels like a passing bus

and we stay here and there 
it goes and leaves."

the answer would be:

1: 1 2 1 1 1 1 1 
2: 1 2 3 2 1 2 1 

3: 1 1 1 2 1 2 
4: 1 2 1 3

1. stanza: 8 - 12
2. stanza: 8 - 7

Here is the poem: 


Mit gelben Birnen hänget
Und voll mit wilden Rosen
Das Land in den See,
Ihr holden Schwäne,
Und trunken von Küssen
Tunkt ihr das Haupt
Ins heilignüchterne Wasser.

Weh mir, wo nehm’ ich, wenn
Es Winter ist, die Blumen, und wo
Den Sonnenschein,
Und Schatten der Erde?
Die Mauern stehn
Sprachlos und kalt, im Winde
Klirren die Fahnen. 




In [13]:
%%ai gpt4o
{prompt}

1: 1 2 2 
2: 1 1 1 1 2 
3: 1 1 1 2 
4: 1 2 2 
5: 1 2 2 
6: 1 1 1 
7: 1 6 3 2 

8: 1 1 1 1 
9: 1 2 1 2 1 
10: 1 3 2 
11: 1 2 1 2 
12: 1 2 1 
13: 1 1 1 1 1 2 
14: 2 1 2 

1. stanza: 15 - 16 
2. stanza: 9 - 7 - 6 - 5

In [14]:
%%ai opus
{prompt}

Here is the vowel count for each word in the poem:

1: 1 2 2 2
2: 1 1 1 2 2
3: 1 1 1 1 1
4: 1 2 2
5: 1 2 1 2
6: 1 1 1 1
7: 1 5 2

8: 1 1 1 1 1 1
9: 1 2 1 1 2 1 1
10: 1 3
11: 1 2 1 2
12: 1 2 1
13: 2 1 1 1 2
14: 2 1 2

1. stanza: 9 - 8 - 5 - 5 - 6 - 4 - 8
2. stanza: 6 - 9 - 4 - 6 - 4 - 7 - 5

In [16]:
print(gemini({prompt}))

Here is the vowel count for your poem:

1: 2 2 2 2 2 
2: 1 3 1 2 2 
3: 1 3 1 2 1
4: 2 3 2 1
5: 1 3 2 1 2
6: 2 1 1 1 2
7: 1 3 3 2 1.

8: 1 2 1 2 1 1 
9: 1 2 2 1 1 1
10: 1 3 2 2 2
11: 1 2 2 1 1 
12: 1 2 2 2 
13: 2 2 1 1 1 
14: 2 2 2 1

1. Stanza: 11 - 11
2. Stanza: 9 - 11 



No, they cannot.

# Results and Discussion

The metrical analysis of both poems shows across the models a surprisingly high amount of errors. The answers also show, that the models have difficulties to remember their own results. These errors are not caused by the fact that we analyze German poems, but can also be seen in a quite famous English poem. 